In [1]:
import polars as pl
from pprint import pprint

label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}

train_df = pl.read_parquet('train-00000-of-00001.parquet')
sampled = train_df.sample(n=50, seed=42)
sampled.head()

cid,fid,premise,hypothesis,verifiable,fever_gold_label,label
i64,str,str,str,i64,str,i64
157554,"""4f89bf1e-e4d7-491a-9f88-8227c2…","""Danny Glover has yet to appear…","""Danny Glover . He also has pro…",1,"""SUPPORTS""",0
222589,"""ff0ad208-2b29-46db-9120-fc644a…","""Same-sex marriage remains ille…","""Same-sex marriage . , same-sex…",1,"""REFUTES""",2
119844,"""06a4345f-e3d9-4c38-8ff2-4ff7cf…","""The United Kingdom is the 21st…","""The United Kingdom of Great Br…",1,"""REFUTES""",2
38234,"""69079a37-7d57-4825-929f-ad9940…","""Roger Penrose has received pri…","""Roger Penrose . He has receive…",1,"""SUPPORTS""",0
71600,"""d3864ece-3e01-4bc1-9b75-b107f2…","""Star Wars: The Last Jedi is a …","""Laura Dern . Her other film ro…",1,"""SUPPORTS""",0


In [2]:
def get_prediction(claim:str):
    return {'entailment' : 0.5, 'neutral' : 0.3, 'contradiction' : 0.2}

In [4]:
for row in sampled.iter_rows(named=True):
    print("PREMISE\n", row['hypothesis'])
    print("CLAIM:\n", row['premise'])
    print("GOLD LABEL:", label_map[row["label"]])
    print("-"*30)
    while True:
        claim = input("> write new claim: ")
        prediction = get_prediction(claim)
        predicted = max(prediction, key=prediction.get)
        if predicted != label_map[row["label"]]:
            print(f"PREDICTED LABEL CHANGED: {predicted} -- {prediction}", flush=True)
        else:
            print(f"SAME PREDICTED LABEL: {predicted} -- {prediction}", flush=True)
        flag = input("type n to exit, otherwise push enter to retry ")
        if flag.lower() == 'n':
            break
    break

PREMISE
 Danny Glover . He also has prominent supporting roles in Silverado ( 1985 ) , Witness ( 1985 ) , Predator 2 ( 1990 ) , Saw ( 2004 ) , Shooter ( 2007 ) , 2012 ( 2009 ) , Death at a Funeral ( 2010 ) , Beyond the Lights ( 2014 ) , and Dirty Grandpa ( 2016 ) . Mumford is a 1999 American comedy-drama film written and directed by Lawrence Kasdan . Silverado is a 1985 American western film produced and directed by Lawrence Kasdan , written by Kasdan and his brother Mark .
CLAIM:
 Danny Glover has yet to appear in a film written by Lawrence Kasdan and his brother Mark.
GOLD LABEL: entailment
------------------------------
SAME PREDICTED LABEL: entailment -- {'entailment': 0.5, 'neutral': 0.3, 'contradiction': 0.2}
SAME PREDICTED LABEL: entailment -- {'entailment': 0.5, 'neutral': 0.3, 'contradiction': 0.2}
SAME PREDICTED LABEL: entailment -- {'entailment': 0.5, 'neutral': 0.3, 'contradiction': 0.2}
SAME PREDICTED LABEL: entailment -- {'entailment': 0.5, 'neutral': 0.3, 'contradiction'

In [ ]:
choosen_model = random.choice(list(models.keys()))
print(f"Choosen Model: {choosen_model}")

premise = input("Insert premise:")
hypothesis = input("Insert hypothesis:")
label = input("Insert gold label (entailment, neutral, contraddiction):")

while True:
    model_input = tokenizers[choosen_model](premise, hypothesis, truncation=True, return_tensors="pt")
    output = models[choosen_model](model_input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = ["entailment", "neutral", "contradiction"]
    prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}

    print(f"Gold label: {label}")
    print(f"Predicted label: {max(prediction, key=prediction.get)}")
    print(f"Predicted Percentage: {prediction}")

    c = input("continue? (Y|n)")

    if c ! = "Y":
        break

    hypothesis = input("Insert hypothesis:")